In [2]:
# Cell 1: Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import os

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.13.0


## Configuration

In [3]:
# Cell 2: Configuration
# MNIST has 60,000 train + 10,000 test = 70,000 total samples
# With 64 train + 16 test per client, max possible clients = 625 (limited by test data)
# Using 100 clients = 6,400 train + 1,600 test samples (~10% of MNIST)

NUM_CLIENTS = 100         # Number of clients (max ~625 with unique data)
NUM_ROUNDS = 10           # Communication rounds
SAMPLES_PER_CLIENT = 80   # Total samples per client (64 train + 16 test)
TRAIN_SAMPLES = 64        # Training samples per client (80% of data)
TEST_SAMPLES = 16         # Test samples per client (20% of data)
CHECK_INTERVAL = 5        # Check accuracy every N epochs
MAX_EPOCHS = 50           # Max epochs per round (reduced for faster training)
BATCH_SIZE = 16           # Smaller batch size for small data

print("=" * 60)
print("FEDERATED LEARNING WITH WEIGHT REJECTION (SMALL DATA)")
print("=" * 60)
print(f"Number of Clients: {NUM_CLIENTS}")
print(f"Training Samples per Client: {TRAIN_SAMPLES}")
print(f"Test Samples per Client: {TEST_SAMPLES}")
print(f"Total Data Used: {NUM_CLIENTS * TRAIN_SAMPLES} train + {NUM_CLIENTS * TEST_SAMPLES} test")
print(f"Communication Rounds: {NUM_ROUNDS}")
print(f"Check Interval: Every {CHECK_INTERVAL} epochs")
print(f"Max Epochs per Round: {MAX_EPOCHS}")
print("=" * 60 + "\n")

FEDERATED LEARNING WITH WEIGHT REJECTION (SMALL DATA)
Number of Clients: 100
Training Samples per Client: 64
Test Samples per Client: 16
Total Data Used: 6400 train + 1600 test
Communication Rounds: 10
Check Interval: Every 5 epochs
Max Epochs per Round: 50



## Data Preparation

In [4]:
# Cell 3: Load MNIST and create small splits
print("Loading MNIST dataset...")
(x_train_full, y_train_full), (x_test_full, y_test_full) = mnist.load_data()

# Normalize
x_train_full = x_train_full / 255.0
x_test_full = x_test_full / 255.0

# Flatten
x_train_full = x_train_full.reshape(len(x_train_full), 28*28)
x_test_full = x_test_full.reshape(len(x_test_full), 28*28)

print(f"Original MNIST - Train: {len(x_train_full)}, Test: {len(x_test_full)}")

# Create small data splits for each client
print(f"\nCreating {NUM_CLIENTS} client data splits...")
client_data = []
np.random.seed(42)

for i in range(NUM_CLIENTS):
    # Randomly sample train and test data
    train_indices = np.random.choice(len(x_train_full), TRAIN_SAMPLES, replace=False)
    test_indices = np.random.choice(len(x_test_full), TEST_SAMPLES, replace=False)
    
    client_data.append({
        'x_train': x_train_full[train_indices],
        'y_train': y_train_full[train_indices],
        'x_test': x_test_full[test_indices],
        'y_test': y_test_full[test_indices]
    })
    
    print(f"✓ Client {i+1} - Train: {TRAIN_SAMPLES}, Test: {TEST_SAMPLES}")

print(f"\n✓ All {NUM_CLIENTS} clients ready\n")

Loading MNIST dataset...
11490434/11490434 [==============================] - 6s 1us/step
Original MNIST - Train: 60000, Test: 10000

Creating 100 client data splits...
✓ Client 1 - Train: 64, Test: 16
✓ Client 2 - Train: 64, Test: 16
✓ Client 3 - Train: 64, Test: 16
✓ Client 4 - Train: 64, Test: 16
✓ Client 5 - Train: 64, Test: 16
✓ Client 6 - Train: 64, Test: 16
✓ Client 7 - Train: 64, Test: 16
✓ Client 8 - Train: 64, Test: 16
✓ Client 9 - Train: 64, Test: 16
✓ Client 10 - Train: 64, Test: 16
✓ Client 11 - Train: 64, Test: 16
✓ Client 12 - Train: 64, Test: 16
✓ Client 13 - Train: 64, Test: 16
✓ Client 14 - Train: 64, Test: 16
✓ Client 15 - Train: 64, Test: 16
✓ Client 16 - Train: 64, Test: 16
✓ Client 17 - Train: 64, Test: 16
✓ Client 18 - Train: 64, Test: 16
✓ Client 19 - Train: 64, Test: 16
✓ Client 20 - Train: 64, Test: 16
✓ Client 21 - Train: 64, Test: 16
✓ Client 22 - Train: 64, Test: 16
✓ Client 23 - Train: 64, Test: 16
✓ Client 24 - Train: 64, Test: 16
✓ Client 25 - Train: 64,

## Model Architecture

In [5]:
# Cell 4: Define lightweight model (optimized for small data)
def create_model():
    """Lightweight model optimized for small datasets"""
    model = keras.Sequential([
        keras.layers.Dense(64, input_shape=(784,), activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation="softmax")
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Test model creation
print("Testing model architecture...")
test_model = create_model()
test_model.summary()
print("✓ Model architecture validated\n")

Testing model architecture...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                50240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 52650 (205.66 KB)
Trainable params: 52650 (205.66 KB)
Non-trainable params: 0 (0.00 Byte)
______________________________________________________

## Federated Learning Functions

In [6]:
# Cell 5: Federated averaging function
def average_weights(weights_list):
    """Average weights from all clients"""
    return [np.mean(weights_tuple, axis=0) for weights_tuple in zip(*weights_list)]

def train_client_with_rejection(client_model, client_data, prev_accuracy, round_num):
    """
    Train client with weight rejection mechanism
    Returns: best_weights, final_accuracy, epochs_used, rejections
    """
    x_train = client_data['x_train']
    y_train = client_data['y_train']
    x_test = client_data['x_test']
    y_test = client_data['y_test']
    
    best_weights = [w.copy() for w in client_model.get_weights()]
    best_accuracy = prev_accuracy
    total_epochs = 0
    rejections = 0
    
    # First round: establish baseline
    if round_num == 0:
        history = client_model.fit(
            x_train, y_train,
            epochs=CHECK_INTERVAL,
            batch_size=BATCH_SIZE,
            validation_split=0.15,
            verbose=0
        )
        total_epochs = CHECK_INTERVAL
        _, current_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
        best_weights = [w.copy() for w in client_model.get_weights()]
        best_accuracy = current_accuracy
        
        return best_weights, best_accuracy, total_epochs, 0
    
    # Subsequent rounds: train with rejection
    for epoch_checkpoint in range(CHECK_INTERVAL, MAX_EPOCHS + 1, CHECK_INTERVAL):
        client_model.fit(
            x_train, y_train,
            epochs=CHECK_INTERVAL,
            batch_size=BATCH_SIZE,
            validation_split=0.15,
            verbose=0
        )
        
        total_epochs += CHECK_INTERVAL
        _, current_accuracy = client_model.evaluate(x_test, y_test, verbose=0)
        
        # Accept if improved
        if current_accuracy > best_accuracy:
            best_weights = [w.copy() for w in client_model.get_weights()]
            best_accuracy = current_accuracy
            break
        else:
            rejections += 1
            if total_epochs >= MAX_EPOCHS:
                # Revert to best weights
                client_model.set_weights(best_weights)
                break
    
    return best_weights, best_accuracy, total_epochs, rejections

print("✓ Federated learning functions defined\n")

✓ Federated learning functions defined



## Training Loop

In [7]:
# Cell 6: Initialize and train
print("=" * 60)
print("INITIALIZING FEDERATED LEARNING")
print("=" * 60)

# Initialize global model
global_model = create_model()
global_weights = global_model.get_weights()

# Tracking variables
client_accuracies = {f'client_{i}': [] for i in range(NUM_CLIENTS)}
client_prev_accuracy = {f'client_{i}': 0.0 for i in range(NUM_CLIENTS)}
client_best_weights = {f'client_{i}': None for i in range(NUM_CLIENTS)}
client_epochs_used = {f'client_{i}': [] for i in range(NUM_CLIENTS)}
client_rejections = {f'client_{i}': [] for i in range(NUM_CLIENTS)}

print("\n" + "=" * 60)
print("STARTING FEDERATED TRAINING")
print("=" * 60 + "\n")

# Main training loop
for round_num in range(NUM_ROUNDS):
    print(f"\n{'='*60}")
    print(f"ROUND {round_num + 1}/{NUM_ROUNDS}")
    print(f"{'='*60}\n")
    
    local_weights = []
    
    for client_id in range(NUM_CLIENTS):
        # Create client model
        client_model = create_model()
        
        # Load previous best weights or global weights
        if client_best_weights[f'client_{client_id}'] is not None:
            client_model.set_weights(client_best_weights[f'client_{client_id}'])
        else:
            client_model.set_weights(global_weights)
        
        # Train with rejection
        prev_acc = client_prev_accuracy[f'client_{client_id}']
        best_weights, final_acc, epochs, rejections = train_client_with_rejection(
            client_model, client_data[client_id], prev_acc, round_num
        )
        
        # Update tracking
        client_best_weights[f'client_{client_id}'] = best_weights
        client_prev_accuracy[f'client_{client_id}'] = final_acc
        client_accuracies[f'client_{client_id}'].append(final_acc)
        client_epochs_used[f'client_{client_id}'].append(epochs)
        client_rejections[f'client_{client_id}'].append(rejections)
        
        local_weights.append(best_weights)
        
        # Print client results
        status = "✓ BASELINE" if round_num == 0 else f"{'✓ IMPROVED' if final_acc > prev_acc else '→ KEPT PREVIOUS'}"
        print(f"Client {client_id+1}: Acc={final_acc*100:.2f}% | Epochs={epochs} | Rejections={rejections} {status}")
    
    # Federated averaging
    global_weights = average_weights(local_weights)
    global_model.set_weights(global_weights)
    
    # Round summary
    round_accs = [client_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
    print(f"\n📊 Round {round_num+1} Avg Accuracy: {np.mean(round_accs):.2f}% (Min: {np.min(round_accs):.2f}%, Max: {np.max(round_accs):.2f}%)")

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60 + "\n")

INITIALIZING FEDERATED LEARNING

STARTING FEDERATED TRAINING


ROUND 1/10

Client 1: Acc=12.50% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 2: Acc=37.50% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 3: Acc=18.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 4: Acc=31.25% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 5: Acc=31.25% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 6: Acc=31.25% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 7: Acc=43.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 8: Acc=37.50% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 9: Acc=18.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 10: Acc=43.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 11: Acc=43.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 12: Acc=18.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 13: Acc=43.75% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 14: Acc=12.50% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 15: Acc=31.25% | Epochs=5 | Rejections=0 ✓ BASELINE
Client 16: Acc=18.75% | Epochs=5 | Rejections=0 ✓

## Results Summary

In [8]:
# Cell 7: Display final results
print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)
print(f"{'Client':<10} {'Final Acc':<15} {'Total Epochs':<15} {'Total Rejections'}")
print("-"*80)

for client_id in range(NUM_CLIENTS):
    final_acc = client_accuracies[f'client_{client_id}'][-1] * 100
    total_epochs = sum(client_epochs_used[f'client_{client_id}'])
    total_rejections = sum(client_rejections[f'client_{client_id}'])
    print(f"{client_id+1:<10} {final_acc:.2f}%{'':<9} {total_epochs:<15} {total_rejections}")

print("-"*80)

# Overall statistics
all_final_accs = [client_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
all_total_epochs = [sum(client_epochs_used[f'client_{i}']) for i in range(NUM_CLIENTS)]
all_total_rejections = [sum(client_rejections[f'client_{i}']) for i in range(NUM_CLIENTS)]

print(f"\n📊 OVERALL SUMMARY:")
print(f"   ▸ Average Final Accuracy: {np.mean(all_final_accs):.2f}%")
print(f"   ▸ Std Dev: {np.std(all_final_accs):.2f}%")
print(f"   ▸ Total Training Epochs: {sum(all_total_epochs)}")
print(f"   ▸ Total Weight Rejections: {sum(all_total_rejections)}")
print("="*80 + "\n")


FINAL RESULTS
Client     Final Acc       Total Epochs    Total Rejections
--------------------------------------------------------------------------------
1          87.50%          230             39
2          75.00%          360             68
3          56.25%          285             52
4          62.50%          320             59
5          62.50%          310             57
6          87.50%          280             51
7          75.00%          325             61
8          56.25%          325             61
9          62.50%          330             62
10         81.25%          385             74
11         81.25%          360             68
12         68.75%          370             70
13         75.00%          335             63
14         43.75%          280             51
15         62.50%          375             72
16         50.00%          335             63
17         68.75%          285             52
18         81.25%          325             61
19         62.50

## Visualization

In [9]:
# Cell 8: Create visualizations
print("Generating visualizations...\n")

# Create output directory
os.makedirs('result_small_data/plots', exist_ok=True)

# Set style
plt.style.use('default')
colors = plt.cm.tab10(np.linspace(0, 1, NUM_CLIENTS))

# ────────────────────────────────────────────
# Plot 1: All Clients Accuracy Over Rounds
# ────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))

for client_id in range(NUM_CLIENTS):
    accs = [acc * 100 for acc in client_accuracies[f'client_{client_id}']]
    ax.plot(range(1, NUM_ROUNDS + 1), accs, 
            marker='o', linewidth=2, markersize=7,
            color=colors[client_id], label=f'Client {client_id+1}')

ax.set_title('Test Accuracy vs Communication Rounds (All Clients)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Communication Round', fontsize=12)
ax.set_ylabel('Test Accuracy (%)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xticks(range(1, NUM_ROUNDS + 1))

plt.tight_layout()
plt.savefig('result_small_data/plots/all_clients_accuracy.png', dpi=300, bbox_inches='tight')
print("✓ Saved: all_clients_accuracy.png")
plt.close()

# ────────────────────────────────────────────
# Plot 2: Average Accuracy with Confidence Band
# ────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))

avg_accs = []
std_accs = []
for round_num in range(NUM_ROUNDS):
    round_accs = [client_accuracies[f'client_{i}'][round_num] * 100 for i in range(NUM_CLIENTS)]
    avg_accs.append(np.mean(round_accs))
    std_accs.append(np.std(round_accs))

rounds = range(1, NUM_ROUNDS + 1)
ax.plot(rounds, avg_accs, marker='s', linewidth=3, markersize=8,
        color='darkblue', label=f'Average (n={NUM_CLIENTS})')
ax.fill_between(rounds,
                np.array(avg_accs) - np.array(std_accs),
                np.array(avg_accs) + np.array(std_accs),
                alpha=0.25, color='darkblue', label='±1 Std Dev')

# Add value labels
for i, acc in enumerate(avg_accs):
    ax.text(i + 1, acc + 0.5, f'{acc:.1f}%', 
            ha='center', fontsize=9, fontweight='bold')

ax.set_title('Average Test Accuracy vs Communication Rounds', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Communication Round', fontsize=12)
ax.set_ylabel('Test Accuracy (%)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xticks(range(1, NUM_ROUNDS + 1))

plt.tight_layout()
plt.savefig('result_small_data/plots/average_accuracy.png', dpi=300, bbox_inches='tight')
print("✓ Saved: average_accuracy.png")
plt.close()

# ────────────────────────────────────────────
# Plot 3: Epochs Used per Client per Round
# ────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 6))

epochs_matrix = np.zeros((NUM_CLIENTS, NUM_ROUNDS))
for client_id in range(NUM_CLIENTS):
    for round_num in range(NUM_ROUNDS):
        epochs_matrix[client_id, round_num] = client_epochs_used[f'client_{client_id}'][round_num]

im = ax.imshow(epochs_matrix, cmap='YlOrRd', aspect='auto')
cbar = plt.colorbar(im, ax=ax, label='Epochs Used')

# Add text annotations
for i in range(NUM_CLIENTS):
    for j in range(NUM_ROUNDS):
        text = ax.text(j, i, f'{int(epochs_matrix[i, j])}',
                      ha='center', va='center', fontsize=10, fontweight='bold',
                      color='white' if epochs_matrix[i, j] > MAX_EPOCHS/2 else 'black')

ax.set_title('Epochs Used per Client per Round', fontsize=14, fontweight='bold')
ax.set_xlabel('Communication Round', fontsize=12)
ax.set_ylabel('Client', fontsize=12)
ax.set_xticks(range(NUM_ROUNDS))
ax.set_xticklabels(range(1, NUM_ROUNDS + 1))
ax.set_yticks(range(NUM_CLIENTS))
ax.set_yticklabels([f'Client {i+1}' for i in range(NUM_CLIENTS)])

plt.tight_layout()
plt.savefig('result_small_data/plots/epochs_heatmap.png', dpi=300, bbox_inches='tight')
print("✓ Saved: epochs_heatmap.png")
plt.close()

# ────────────────────────────────────────────
# Plot 4: Weight Rejections Heatmap
# ────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 6))

rejections_matrix = np.zeros((NUM_CLIENTS, NUM_ROUNDS))
for client_id in range(NUM_CLIENTS):
    for round_num in range(NUM_ROUNDS):
        rejections_matrix[client_id, round_num] = client_rejections[f'client_{client_id}'][round_num]

im = ax.imshow(rejections_matrix, cmap='Reds', aspect='auto')
cbar = plt.colorbar(im, ax=ax, label='Rejections')

# Add text annotations
for i in range(NUM_CLIENTS):
    for j in range(NUM_ROUNDS):
        text = ax.text(j, i, f'{int(rejections_matrix[i, j])}',
                      ha='center', va='center', fontsize=10, fontweight='bold',
                      color='white' if rejections_matrix[i, j] > 3 else 'black')

ax.set_title('Weight Rejections per Client per Round', fontsize=14, fontweight='bold')
ax.set_xlabel('Communication Round', fontsize=12)
ax.set_ylabel('Client', fontsize=12)
ax.set_xticks(range(NUM_ROUNDS))
ax.set_xticklabels(range(1, NUM_ROUNDS + 1))
ax.set_yticks(range(NUM_CLIENTS))
ax.set_yticklabels([f'Client {i+1}' for i in range(NUM_CLIENTS)])

plt.tight_layout()
plt.savefig('result_small_data/plots/rejections_heatmap.png', dpi=300, bbox_inches='tight')
print("✓ Saved: rejections_heatmap.png")
plt.close()

# ────────────────────────────────────────────
# Plot 5: Summary Bar Chart
# ────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Final accuracies
clients = [f'C{i+1}' for i in range(NUM_CLIENTS)]
final_accs = [client_accuracies[f'client_{i}'][-1] * 100 for i in range(NUM_CLIENTS)]
bars1 = ax1.bar(clients, final_accs, color=colors, edgecolor='black', linewidth=1.5)

for bar, acc in zip(bars1, final_accs):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{acc:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax1.set_title('Final Test Accuracy per Client', fontsize=13, fontweight='bold')
ax1.set_ylabel('Accuracy (%)', fontsize=11)
ax1.set_xlabel('Client', fontsize=11)
ax1.grid(axis='y', alpha=0.3, linestyle='--')
ax1.set_ylim([0, 100])

# Total epochs
total_epochs_list = [sum(client_epochs_used[f'client_{i}']) for i in range(NUM_CLIENTS)]
bars2 = ax2.bar(clients, total_epochs_list, color=colors, edgecolor='black', linewidth=1.5)

for bar, epochs in zip(bars2, total_epochs_list):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{epochs}', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax2.set_title('Total Epochs Used per Client', fontsize=13, fontweight='bold')
ax2.set_ylabel('Total Epochs', fontsize=11)
ax2.set_xlabel('Client', fontsize=11)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('result_small_data/plots/summary_bars.png', dpi=300, bbox_inches='tight')
print("✓ Saved: summary_bars.png")
plt.close()

print("\n✅ All visualizations generated!\n")

Generating visualizations...



C:\Users\Dell\AppData\Local\Temp\ipykernel_28132\545474026.py:30: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


✓ Saved: all_clients_accuracy.png
✓ Saved: average_accuracy.png
✓ Saved: epochs_heatmap.png
✓ Saved: rejections_heatmap.png
✓ Saved: summary_bars.png

✅ All visualizations generated!



## Individual Client Accuracy Progressions

In [11]:
# Cell 8b: Create neat grid of individual client accuracy progressions
print("Creating individual client accuracy progression diagrams...\n")

# Calculate grid dimensions
n_cols = 5
n_rows = int(np.ceil(NUM_CLIENTS / n_cols))

# Create figure with subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 2.5))
fig.suptitle('Accuracy Progression for Each Client Across Communication Rounds', 
             fontsize=16, fontweight='bold', y=0.995)

# Flatten axes for easier iteration
axes = axes.flatten() if NUM_CLIENTS > 1 else [axes]

# Plot each client
for client_id in range(NUM_CLIENTS):
    ax = axes[client_id]
    
    # Get accuracy data
    accs = [client_accuracies[f'client_{client_id}'][round_num] * 100 
            for round_num in range(NUM_ROUNDS)]
    rounds = list(range(1, NUM_ROUNDS + 1))
    
    # Plot line
    ax.plot(rounds, accs, marker='o', linewidth=2.5, markersize=6,
            color='#2E86AB', markerfacecolor='#A23B72', markeredgewidth=2, 
            markeredgecolor='#2E86AB')
    
    # Fill area under curve
    ax.fill_between(rounds, accs, alpha=0.2, color='#2E86AB')
    
    # Add start and end accuracy labels
    ax.text(1, accs[0], f'{accs[0]:.1f}%', ha='right', va='bottom', 
            fontsize=8, fontweight='bold', color='#A23B72')
    ax.text(NUM_ROUNDS, accs[-1], f'{accs[-1]:.1f}%', ha='left', va='bottom', 
            fontsize=8, fontweight='bold', color='#A23B72')
    
    # Calculate improvement
    improvement = accs[-1] - accs[0]
    improvement_color = 'green' if improvement >= 0 else 'red'
    
    # Title with client number and improvement
    ax.set_title(f'Client {client_id + 1} ({improvement:+.1f}%)', 
                fontsize=10, fontweight='bold', color=improvement_color)
    
    # Formatting
    ax.set_xlabel('Round', fontsize=8)
    ax.set_ylabel('Accuracy (%)', fontsize=8)
    ax.grid(True, alpha=0.25, linestyle='--', linewidth=0.5)
    ax.set_xticks(rounds)
    ax.set_ylim([0, 100])
    ax.tick_params(labelsize=7)

# Hide unused subplots
for idx in range(NUM_CLIENTS, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('result_small_data/plots/individual_client_grid.png', dpi=300, bbox_inches='tight')
print("✓ Saved: individual_client_grid.png")
plt.close()

# ────────────────────────────────────────────
# Alternative: Detailed view for selected clients (top 10)
# ────────────────────────────────────────────
print("\nCreating detailed view of top 10 performing clients...\n")

# Get top 10 clients by final accuracy
final_accs_with_id = [(i, client_accuracies[f'client_{i}'][-1] * 100) 
                       for i in range(NUM_CLIENTS)]
top_10_clients = sorted(final_accs_with_id, key=lambda x: x[1], reverse=True)[:10]

fig, ax = plt.subplots(figsize=(14, 7))

for rank, (client_id, final_acc) in enumerate(top_10_clients):
    accs = [client_accuracies[f'client_{client_id}'][round_num] * 100 
            for round_num in range(NUM_ROUNDS)]
    rounds = list(range(1, NUM_ROUNDS + 1))
    
    color = plt.cm.viridis(rank / 10)
    ax.plot(rounds, accs, marker='o', linewidth=2.5, markersize=7,
            color=color, label=f'Client {client_id + 1} (Final: {final_acc:.1f}%)',
            alpha=0.8)

ax.set_title('Top 10 Performing Clients - Accuracy Progression', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Communication Round', fontsize=13)
ax.set_ylabel('Test Accuracy (%)', fontsize=13)
ax.legend(loc='best', fontsize=9, ncol=2)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xticks(range(1, NUM_ROUNDS + 1))
ax.set_ylim([0, 100])

plt.tight_layout()
plt.savefig('result_small_data/plots/top_10_clients_detailed.png', dpi=300, bbox_inches='tight')
print("✓ Saved: top_10_clients_detailed.png")
plt.close()

print("\n✅ Individual client diagrams generated!\n")

Creating individual client accuracy progression diagrams...

✓ Saved: individual_client_grid.png

Creating detailed view of top 10 performing clients...

✓ Saved: top_10_clients_detailed.png

✅ Individual client diagrams generated!



## Save Results

In [12]:
# Cell 9: Save results and model
print("Saving results...")

# Save training results
results_data = {
    'client_accuracies': client_accuracies,
    'client_epochs_used': client_epochs_used,
    'client_rejections': client_rejections,
    'config': {
        'num_clients': NUM_CLIENTS,
        'num_rounds': NUM_ROUNDS,
        'train_samples': TRAIN_SAMPLES,
        'test_samples': TEST_SAMPLES,
        'check_interval': CHECK_INTERVAL,
        'max_epochs': MAX_EPOCHS,
        'batch_size': BATCH_SIZE
    },
    'summary': {
        'avg_final_accuracy': np.mean(all_final_accs),
        'std_final_accuracy': np.std(all_final_accs),
        'total_epochs': sum(all_total_epochs),
        'total_rejections': sum(all_total_rejections)
    }
}

os.makedirs('result_small_data', exist_ok=True)
np.save('result_small_data/training_results.npy', results_data)
print("✓ Results saved: result_small_data/training_results.npy")

# Save global model
global_model.save('result_small_data/federated_global_model.h5')
print("✓ Model saved: result_small_data/federated_global_model.h5")

print("\n" + "="*80)
print("✅ FEDERATED LEARNING COMPLETE!")
print("="*80)
print(f"\n📁 All results saved in: result_small_data/")
print(f"   ▸ Training results: training_results.npy")
print(f"   ▸ Global model: federated_global_model.h5")
print(f"   ▸ Plots: plots/ directory")
print("\n" + "="*80)

Saving results...
✓ Results saved: result_small_data/training_results.npy
✓ Model saved: result_small_data/federated_global_model.h5

✅ FEDERATED LEARNING COMPLETE!

📁 All results saved in: result_small_data/
   ▸ Training results: training_results.npy
   ▸ Global model: federated_global_model.h5
   ▸ Plots: plots/ directory

